# Spotify Exploratory Data Analysis

## Summary

Spotify is one of the most popular music streaming applications, originally from Sweden, with 406M active users worldwide, founded by Daniel Ek and Martin Lorentzon in 2006. 108M subscribers have subscribed to Spotify Premium, providing an ad-free experience, among other benefits.  As a music lover, I regulary listen to my favorite songs and artists on Spotify - finding different genres of music that fit my mood.  Every year, Spotify provides users a "year in rewind" service to show their "music statistics" from the past year.  I wanted to create my own Spotify Rewind to have a more personalized summary of the music I have listened to so far.


The questions we would like to look at are:

- Who are my top artists I have listened to?
- Who are the top genres I listened to?
- What are the top 5 songs played?
- How many minutes listened by month?

## Step 1 - Request the data

Data was manually requested from Spotify's website.  The data was delivered in separate JSON files within 5 business days of the original request.  The files that I am only going to be focusing on are the ones related to __Streaming History__, as they contain the data needed for analysis.

## Step 2 - Preparation of the streaming and library data

The data colleced will be used to create a single dataframe that includes all streaming data, each song in the library, and each song's Spotify URI (Unique Identifier).

In [1]:
## Import the necessary packages
import pandas as pd
import numpy as np
import requests # This will be needed later for using Spotify's API

In [4]:
## Reading each StreamingHistory files and assigning them initially to their own variable
df_stream0 = pd.read_json('StreamingHistory0.json')
df_stream1 = pd.read_json('StreamingHistory1.json')
df_stream2 = pd.read_json('StreamingHistory2.json')
df_stream3 = pd.read_json('StreamingHistory3.json')

df_stream = pd.concat([df_stream0, df_stream1, df_stream2, df_stream3])


In [8]:
## Taking an inital look at the data: what columns are present 
df_stream.columns

Index(['endTime', 'artistName', 'trackName', 'msPlayed'], dtype='object')

We see that the merged dataframe contains the columns: endTime, artistName, trackName, and msPlayed - all of which are of the type ojbect.

In [10]:
## Create uniqueID for each song through combining fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']
df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2021-11-01 01:04,Terence Lam,難道喜歡處女座 (Alter Ego),142805,Terence Lam:難道喜歡處女座 (Alter Ego)
1,2021-11-01 01:47,2PM,Heartbeat,193398,2PM:Heartbeat
2,2021-11-01 01:55,2PM,She’s ma girl,18670,2PM:She’s ma girl
3,2021-11-01 01:58,AILEE,My Lips,41141,AILEE:My Lips
4,2021-11-01 02:02,ERIK,Ăn Sáng Nha - Original Version,187241,ERIK:Ăn Sáng Nha - Original Version


Next I cleaned the 'YourLibrary' file retrieved from Spotify so that it contained the "tracks" dictionary, surrounded by brackets and saved it as a new file.  This is a JSON file containing the artist, album, track, and uri for a specific track.

In [17]:
import json # Used to parse through the JSON data

# This is to convert the JSON file into a CSV dataframe
with open("YourLibrary.json","r") as fp:
    data = json.load(fp)
    data = data['tracks']
df_library = pd.DataFrame.from_dict(data)

# Adding a uniqueID column 
df_library['UniqueID'] = df_library['artist'] + ':' + df_library['track']
# Add the column with track URI - removing 'spotify:track' as it is not needed
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Younha,혜성,손을 잡고서,spotify:track:4Wbr28lSOrBHGzmg9m7io7,Younha:손을 잡고서,4Wbr28lSOrBHGzmg9m7io7
1,Leehom Wang,蓋世英雄,在梅邊,spotify:track:7tF3CpbFS0kdkXu3zHmA1i,Leehom Wang:在梅邊,7tF3CpbFS0kdkXu3zHmA1i
2,JJ Lin,偉大的渺小,聖所,spotify:track:6ilHdm1oBzsR5lY7vlPFhx,JJ Lin:聖所,6ilHdm1oBzsR5lY7vlPFhx
3,Wanting,Everything In The World,Drenched,spotify:track:5Slyt01qJpBYW2XQbMWk5D,Wanting:Drenched,5Slyt01qJpBYW2XQbMWk5D
4,Joe Inoue,幻,幻,spotify:track:2rEOfZT1isxKmG9FfdULBG,Joe Inoue:幻,2rEOfZT1isxKmG9FfdULBG


Next, I will prepare the final dataframe that will be used to load into Tableau for dashboard creation.

In [19]:
# final dataframe
tableau = df_stream.copy()

# inserting column to see if stream song is in library
tableau['In Library'] = np.where(tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# Performing left join with df_library on UniqueID to bring album and track_uri
tableau = pd.merge(tableau, df_library[['album', 'UniqueID', 'track_uri']], how='left', on=['UniqueID'])

tableau.head()

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,2021-11-01 01:04,Terence Lam,難道喜歡處女座 (Alter Ego),142805,Terence Lam:難道喜歡處女座 (Alter Ego),0,NaN,NaN
1,2021-11-01 01:47,2PM,Heartbeat,193398,2PM:Heartbeat,0,NaN,NaN
2,2021-11-01 01:55,2PM,She’s ma girl,18670,2PM:She’s ma girl,0,NaN,NaN
3,2021-11-01 01:58,AILEE,My Lips,41141,AILEE:My Lips,0,NaN,NaN
4,2021-11-01 02:02,ERIK,Ăn Sáng Nha - Original Version,187241,ERIK:Ăn Sáng Nha - Original Version,0,NaN,NaN


## Step 3 - Creating a New Spotify Project

Next, I retrieved a __'Client ID'__ and __'Client Secret'__ from my Spotify Developer Account.  To protect sensitve data, I have scrubbed my Client ID and Client Secret from the final file.

## Step 4 - Create the genre dataframe with Spotify's API

The __'Client ID'__ and __'Client Secret'__ as previously mentioned will be used to generate an access token in order to pull data from the API.

In [20]:
CLIENT_ID = 'CLIENT ID'
CLIENT_SECRET = 'CLIENT SECRET'

In [22]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST - function used for API data retrieval
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': 'CLIENT ID',
    'client_secret':'CLIENT SECRET'
})

# Convert response to JSON
response_data = auth_response.json()

# Saving access token
access_token = response_data['access_token']

In [23]:
# Authenticate API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [24]:
# URL of all Spotify endpoints
BASE_URL = 'https://api.spotify.com/v1/'

Next, the data for artists and genres associated with each of the track_uri in our library will be pulled and added to a dictionary

In [31]:
# creating blank dictionary to store the data
dict_genre = {}

# convert track_uri column as an iterable list
track_uris = df_library['track_uri'].to_list()

"""
Loop through each of the track URIs and pull artist
URI using the API.  The artist URI will be used to 
pull genres associated with the specific artist, which
will all be stored in a dictionary
"""

for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

This dictionary will be converted to a dataframe and expand so that each genre for each track/artist is in its own line.

In [35]:
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre

,track_uri,artist_uri,genres
0,4Wbr28lSOrBHGzmg9m7io7,6GwM5CHqhWXzG3l5kzRSAS,"[k-pop, korean pop]"
1,7tF3CpbFS0kdkXu3zHmA1i,2F5W6Rsxwzg0plQ0w8dSyt,"[c-pop, mandopop, taiwan pop]"
2,6ilHdm1oBzsR5lY7vlPFhx,7Dx7RhX0mFuXhCOUgB01uM,"[mandopop, singaporean mandopop, singaporean pop]"
3,5Slyt01qJpBYW2XQbMWk5D,2OC4lXfGEKZkbmRCcf2vTq,[mandopop]
4,2rEOfZT1isxKmG9FfdULBG,04q6GxPQU1OTqHGrUxeufd,"[j-poprock, otacore]"
...,...,...,...
624,0mxtd9oVKsT2mZXXWnowup,0sQMt1Llvcuza8oLKB9bmi,[cantopop]
625,0I2We7TVGZhimAXUswSfMl,16s0YTFcyjP4kgFwt7ktrY,"[mandopop, taiwan pop]"
626,6PxEvURvLikKggWw4VgNfJ,7oMkAwPOEfk2ETm9HX9Ruo,"[hakkapop, taiwan singer-songwriter]"
627,7sKxjQ0t5RX2Ds8vgc687j,5YsDFZYpkNgiIxfDW6rXlu,[cantopop]


In [36]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,4Wbr28lSOrBHGzmg9m7io7,6GwM5CHqhWXzG3l5kzRSAS,k-pop
0,4Wbr28lSOrBHGzmg9m7io7,6GwM5CHqhWXzG3l5kzRSAS,korean pop
1,7tF3CpbFS0kdkXu3zHmA1i,2F5W6Rsxwzg0plQ0w8dSyt,c-pop
1,7tF3CpbFS0kdkXu3zHmA1i,2F5W6Rsxwzg0plQ0w8dSyt,mandopop
1,7tF3CpbFS0kdkXu3zHmA1i,2F5W6Rsxwzg0plQ0w8dSyt,taiwan pop


The dataframes __tableau__ and __df_genre_expanded__ will be converted to CSV files to be loaded into Tableau.

In [37]:
tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpanded.csv')
print('done')

done


# Step 5 - Results and Next Steps

The exploratory data analysis was done through a Tableau dashboard (please see the link below)

### Findings

- Top 10 artists listened to (In order from greatest to least): JJ Lin, BTS, Hwa Sa, Jay Chou, Mayday, Hearts Grow, Angela Hui, Joey Young, C AllStar, and Hubert Wu
    - JJ Lin - 1st place with 1,826 minutes listened total. Had the most minutes listened to August with 409.6 minutes.  He had the least minutes listened to 3.8 minutes.

- Top 5 songs: I’m a B (782 Minutes), Permission to Dance (301.8 Minutes), As if it’s your last (297.5 minutes), 再不再見 (Not see again - 296.4 minutes), 夜にかける (Flowing through the night - 285.8 minutes).

- Top Genres: Mandopop + C-pop (664.1 minutes), Canto pop (367.1 minutes), J-pop (108.7)
- Q4 so far has the lowest average number of minutes listened to music on Spotify by 2,349.  This is because Q4 has started.  Accurate numbers will be available by the end of 12/2022.



### Improvements

- One way that this project could be improved is that I could have used Spotify’s API to retrieve the data.  This would have allowed me to gather data within the same day, rather than waiting five business days and manually load the data.  From there, performing ETL, using AWS Redshift and connecting it to the Tableau dashboard so that data would be automatically updated regularly.  As I am using Tableau dashboard and being mindful at potential costs through the AWS service, I have to perform this project manually.

- The Spotify data also picked up podcasts I listed to, which skewed the data.  On average, the podcasts I listen to are on average 40 minutes each.  The data may be skewed as podcasts aren’t music.
- Duplicates in genres.  A noticeable example would be of the “Mandarin Chinese Pop” genre.  In the dataset, they were either reported as “Mandopop” or “C-pop” - in reality, they are the same thing.  Using SQL or Python, I can further normalize the data by standardizing duplicated data.

- I noticed that the genres were mismatched on each artist. A reason to why this may have happened is that tracks that were tied to multiple genres had the values stored in lists. To ensure quality data, I should have parsed through each data point and performed data validation.


Link to Dashboard: https://public.tableau.com/views/SpotifyMusicAnalysis_16674631126220/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link